## Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Environment: Python 3.10.4 (ipykernel)
#### Last Update: 2022-05-18
#### Description: Web crawling code for EURONEXT.com, especially for EURONEXT Amsterdam. Can be reused for other country by simple editing.

### Utilized BS4 and Selenium Web for Crwaling - Visualized by Pandas

In [2]:
import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version

### Environment
#### - Chromdriver must be located in local PC that is running. This must be modified to be used.

In [2]:
path = 'C:\\Users\\DT\\Desktop\\자료\\chromedriver'

In [3]:
print(python_version())

3.10.4


In [4]:
print(pd.__version__)

1.4.2


### Basic setting for csv file that will store 재무정보 data - Order of rows are based on 재무정보 데이터소스

In [5]:
cover = open('DutchData.csv','w', newline='', encoding='CP949')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드', '현지언어국가명','영문국가명','국가언어코드값','현지언어명','영문언어명','현지언어기업명','통화구분코드','화폐단위명','영문기업명','법인등록번호','사업자등록번호','현지언어산업군명','영문산업군명','설립일자','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','회계연도','회계분기','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','이자비용','기타영업외비용금액','영업이익손실금액','이자수익','기타영업외수익금액','법인세차감전순이익','당기순이익/손실금액','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업CAGR','기업 연수','기업 상태','기업추가정보값','작업구분코드','데이터생성일자','연계처리상태코드','연계처리일자'])

438

### Crawled Website
EURONEXT MAIN: https://live.euronext.com/en /
EURONEXT AMSTERDAM: https://live.euronext.com/en/markets/amsterdam/equities/list

## This is the main code that crawls data rom EURONEXT. 
#### - Time sleep was necessary because of comparably long loading time.
#### - Uncollectable or unavailable data was displayed as "(NA) DataName."
#### - (계산예정) Data Will be calculated and inserted manually later.
#### - Sample Visualization is located in the bottom.

In [6]:
#EURONEXT does not allow us to directly move to Amsterdam list. Automated webdriver moves to the list by itself.
base_url = "https://live.euronext.com/"
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/products/equities/list')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#popup-buttons > button.agree-button.eu-cookie-compliance-default-button').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#div-stocks-data-table > button').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#collapsePanelCompactGroupheadingitem1').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#collapsePanelCompactGroup > div:nth-child(1) > div.card-header.panels__header.panels__header--space-between.m-0.bg-transparent > div.panels__column.ml-auto > button > div').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#edit-market-06--2 > div > label').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#edit-awl-pd-filters-submit--2').click()

#Get each company's link, market symbol, and market code on table - P1
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
all_market_maintable = maintable.find_all(attrs={'td',"stocks-market"})
all_symbol_maintable = maintable.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList3 =[]
for a in all_market_maintable :
    newList3.append(a.text)
newList4 =[]
for b in all_symbol_maintable :
    newList4.append(b.text)

#Get each company's link, market symbol, and market code on table - P1
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable2 = soup.find('table', id='stocks-data-table')
time.sleep(2)
all_atag_maintable2 = maintable2.find_all('a')
all_market_maintable2 = maintable2.find_all(attrs={'td',"nowrap pointer"})
all_symbol_maintable2 = maintable2.find_all(attrs={'td',"stocks-symbol"})

#We have to process the collected data again because of their special structure
newList1 =[]
for c in all_market_maintable2 :
    newList1.append(c.text)

newList2 =[]
for d in all_symbol_maintable2 :
    newList2.append(d.text)
    
#Merge two lists (from page 1 and 2)
all_atag_maintable = all_atag_maintable + all_atag_maintable2
all_symbol_maintable = newList3 + newList1
all_market_maintable = newList4 + newList2

#Final Processing to remove unnecessary data
all_symbol_maintable.pop(0)
all_market_maintable.pop(0)
company_links = []
failList = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
#print(len(company_links))

# Main Data Collection code
for sub in company_links:
    driver.get(base_url + sub)
    info = []
    time.sleep(2)
    
    info.append("NLD")
    info.append("Nederland")
    info.append("Nederlands")
    info.append("NL")
    info.append("Nederlands")
    info.append("Dutch")
    info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text)
    info.append("EUR")
    info.append("유로")
    info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text)
    info.append("(NA)법인등록번호")
    info.append("(NA)사업자등록번호")
    try : 
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
        time.sleep(2)
        info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2)').text)
        info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2)').text)
        info.append("(NA) 설립일자")
        info.append(all_symbol_maintable.pop(0))
        info.append("EURONEXT AMSTERDAM")
        info.append("EURONEXT AMSTERDAM")
    except :
        info.append("(NA) 영어산업군명")
        info.append("(NA) 현지언어산업군명")
        info.append("(NA) 설립일자")
        info.append(all_symbol_maintable.pop(0))
        info.append("EURONEXT AMSTERDAM")
        info.append("EURONEXT AMSTERDAM")
        
    info.append(all_market_maintable.pop(0))
    info.append("(NA) 상장일자")
    info.append("(NA) 회계연도")
    info.append("(NA) 회계분기")
    info.append("(NA) 유동자산금액")
    info.append("(NA) 비유동자산금액")
    info.append("(NA) 자산총계금액")
    info.append("(NA) 유동부채금액")
    info.append("(NA) 비유동부채금액")
    info.append("(NA) 부채총계금액")
    info.append("(NA) 자본총계금액")
    info.append("(NA) 부채자본총계금액")
    try :
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
        time.sleep(2)
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "천 유로")
        info.append("(NA) 매출원가금액")
        info.append("(NA) 판매비관리비금액")
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]').text + "천 유로")
        info.append("(NA) 기타영업외비용금액")
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]').text + "천 유로")
        info.append("(NA) 이자수입")
        info.append("(NA) 기타영업외수익금액")
        info.append("(계산예정)법인세차감전순이익")
        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]').text + "천 유로")
    except : 
        info.append("(NA) 매출액")
        info.append("(NA) 매출원가금액")
        info.append("(NA) 판매비관리비금액")
        info.append("(NA) 이자비용")
        info.append("(NA) 기타영업외비용금액")
        info.append("(NA) 영업이익손실금액")
        info.append("(NA) 이자수입")
        info.append("(NA) 기타영업외수익금액")
        info.append("(계산예정)법인세차감전순이익")
        info.append("(NA) 당기순이익")
        failList.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text)
                    
    info.append("(NA) 영업활동현금흐름금액")
    info.append("(NA) 투자활동현금흐름금액")
    info.append("(NA) 재무활동현금흐름금액")
    info.append("(NA) 기초현금자산금액")
    info.append("(NA) 기말현금자산금액")
    info.append("(NA) 부채비율")
                
    info.append("(계산예정)영업이익율")
    info.append("(계산예정)매출액증가율")
    info.append("(계산예정)영업이익증가율")
    info.append("(계산예정)당기순이익 증가율")
    info.append("(계산예정)기업CAGR")
    info.append("(계산예정)기업 연수")        
    info.append("(계산예정)기업 상태")

    try: 
        driver.find_element(By.CSS_SELECTOR, value='#awl-instrument-page-quicktabs-empty > div > div > nav > div > a.nav-item.nav-link.active').click()
        time.sleep(2)
        info.append("주당가치: " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[2]/div/div[1]/div[1]/div/span[2]').text)
        info.append("(NA) 작업구분코드")
        info.append("(NA) 데이터생성일자")
        info.append("(NA) 연계처리상태코드")
        info.append("(NA) 연계처리일자")    
    except :
        info.append("(NA)기업추가정보값")
        info.append("(NA) 작업구분코드")
        info.append("(NA) 데이터생성일자")
        info.append("(NA) 연계처리상태코드")
        info.append("(NA) 연계처리일자")  
    
    #Keep data in csv
    writing.writerow(info);
    
    #Visualize
    print(info)
    
#Failed or not fully crawled company list
print(failList)

C:\Users\DT\AppData\Local\Temp\ipykernel_7352\1799252625.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'AALBERTS N.V.', 'EUR', '유로', 'AALBERTS N.V.', '(NA)법인등록번호', '(NA)사업자등록번호', '5020, Industrial Goods and Services', '5020, Industrial Goods and Services', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'AALB', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '2.979.100천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '13.400천 유로', '(NA) 기타영업외비용금액', '506.100천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '367.800천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 45.00', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'ABN AMRO BANK N.V.', 'EUR

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'AJAX', 'EUR', '유로', 'AJAX', '(NA)법인등록번호', '(NA)사업자등록번호', '4050, Travel and Leisure', '4050, Travel and Leisure', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'AJAX', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '125.159천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '1.818천 유로', '(NA) 기타영업외비용금액', '-7.590천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '-8.105천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 12.30', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'AKZO NOBEL', 'EUR', '유로', 'AKZO NOBEL', '(NA)법인등록번호', '(NA)사업자등록번호', '

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'ARCELORMITTAL SA', 'EUR', '유로', 'ARCELORMITTAL SA', '(NA)법인등록번호', '(NA)사업자등록번호', '5510, Basic Resources', '5510, Basic Resources', '(NA) 설립일자', 'XAMS, XPAR', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'MT', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '67.618.333천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '245.496천 유로', '(NA) 기타영업외비용금액', '14.991.169천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '13.745.143천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 27.13', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'ASM International N.V.', 'EUR', '유로

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'BE Semiconductor Industries N.V.', 'EUR', '유로', 'BE Semiconductor Industries N.V.', '(NA)법인등록번호', '(NA)사업자등록번호', '1010, Technology', '1010, Technology', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'BESI', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '749.297천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '10.724천 유로', '(NA) 기타영업외비용금액', '317.583천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '282.419천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 52.52', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'BenevolentAI', 'EUR', '유로',

...

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'RA SPECIAL ACQUISITION SHARES', 'EUR', '유로', 'RA SPECIAL ACQUISITION SHARES', '(NA)법인등록번호', '(NA)사업자등록번호', '3020, Financial Services', '3020, Financial Services', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'PRX', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '0천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '0천 유로', '(NA) 기타영업외비용금액', '-77천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '-77천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 10.00', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'RA SPECIAL ACQUISITION UNITS', 'EUR',

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'SBM OFFSHORE', 'EUR', '유로', 'SBM OFFSHORE', '(NA)법인등록번호', '(NA)사업자등록번호', '6010, Energy', '6010, Energy', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'SGO', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '3.308.901천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '266.690천 유로', '(NA) 기타영업외비용금액', '648.181천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '416.814천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 14.425', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'SHELL PLC', 'EUR', '유로', 'SHELL PLC', '(NA)법인등록번호', '(NA)사업자등록번호', '6010

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'Thunderbird Resorts Inc.', 'EUR', '유로', 'Thunderbird Resorts Inc.', '(NA)법인등록번호', '(NA)사업자등록번호', '4050, Travel and Leisure', '4050, Travel and Leisure', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'TFG', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '13.289천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '-1.491천 유로', '(NA) 기타영업외비용금액', '298천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '-1.596천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 0.06', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'TIE KINETIX', 'EUR', '유로', 'TIE KIN

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'VAM INVESTMENTS SHARES', 'EUR', '유로', 'VAM INVESTMENTS SHARES', '(NA)법인등록번호', '(NA)사업자등록번호', '3020, Financial Services', '3020, Financial Services', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'PREVA', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '(NA) 매출액', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '(NA) 이자비용', '(NA) 기타영업외비용금액', '(NA) 영업이익손실금액', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '(NA) 당기순이익', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 9.60', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'VAM INVESTMENTS TREASURY SHARES', 

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'WOLTERS KLUWER', 'EUR', '유로', 'WOLTERS KLUWER', '(NA)법인등록번호', '(NA)사업자등록번호', '4030, Media', '4030, Media', '(NA) 설립일자', 'XAMS', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', 'WHA', '(NA) 상장일자', '(NA) 회계연도', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '4.771.000천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '54.000천 유로', '(NA) 기타영업외비용금액', '1.012.000천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '728.000천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당가치: 91.54', '(NA) 작업구분코드', '(NA) 데이터생성일자', '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'YATRA CAPITAL LTD', 'EUR', '유로', 'YATRA CAPITAL LTD', '(NA)법인등록번호', '(

## Data Visualization - Text Version (Sample)

##### Standard

In [6]:
df = pd.read_csv (r'C:\Users\DT\Desktop\자료\DutchDataSample.csv')
print (df)

    헤브론스타국가코드    현지언어국가명       영문국가명 국가언어코드값       현지언어명  영문언어명  \
0         NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
1         NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
2         NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
3         NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
4         NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
..        ...        ...         ...     ...         ...    ...   
171       NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
172       NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
173       NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
174       NLD  Nederland  Nederlands      NL  Nederlands  Dutch   
175       NLD  Nederland  Nederlands      NL  Nederlands  Dutch   

                                               현지언어기업명 통화구분코드 화폐단위명  \
0                                        AALBERTS N.V.    EUR    유로   
1                                   ABN AMRO BANK N.V

##### By rows

In [21]:
with open("DutchData.csv", "r") as table:
    for row in table:
        print(row)

헤브론스타국가코드,현지언어국가명,영문국가명,국가언어코드값,현지언어명,영문언어명,현지언어기업명,통화구분코드,화폐단위명,영문기업명,법인등록번호,사업자등록번호,현지언어산업군명,영문산업군명,설립일자,주식시장코드,현지언어주식시장명,영문주식시장명,상장코드,상장일자,회계연도,회계분기,유동자산금액,비유동자산금액,자산총계금액,유동부채금액,비유동부채금액,부채총계금액,자본총계금액,부채자본총계금액,매출액,매출원가금액,판매비관리비금액,이자비용,기타영업외비용금액,영업이익손실금액,이자수익,기타영업외수익금액,법인세차감전순이익,당기순이익/손실금액,영업활동현금흐름금액,투자활동현금흐름금액,재무활동현금흐름금액,기초현금자산금액,기말현금자산금액,부채비율,영업이익율,매출액증가율,영업이익증가율,당기순이익 증가율,기업CAGR,기업 연수,기업 상태,기업추가정보값,작업구분코드,데이터생성일자,연계처리상태코드,연계처리일자

NLD,Nederland,Nederlands,NL,Nederlands,Dutch,AALBERTS N.V.,EUR,유로,AALBERTS N.V.,(NA)법인등록번호,(NA)사업자등록번호,"5020, Industrial Goods and Services","5020, Industrial Goods and Services",(NA) 설립일자,AALB,EURONEXT AMSTERDAM,EURONEXT AMSTERDAM,XAMS,(NA) 상장일자,(NA) 회계연도,(NA) 회계분기,(NA) 유동자산금액,(NA) 비유동자산금액,(NA) 자산총계금액,(NA) 유동부채금액,(NA) 비유동부채금액,(NA) 부채총계금액,(NA) 자본총계금액,(NA) 부채자본총계금액,2.979.100천 유로,(NA) 매출원가금액,(NA) 판매비관리비금액,13.400천 유로,(NA) 기타영업외비용금액,506.100천 유로,(NA) 이자수입,(NA) 기타영업외수익금액,(계산예정)법인세차감전순이익,367.800천 유로,(NA) 영업활동현금흐름금액,(NA) 투자활동현금흐름금액,(NA) 재무활동현금흐름금액,(NA) 기초현금자산금액,(N

##### Headers

In [8]:
file = open("DutchData.csv")
csvreader = csv.reader(file)
header = next(csvreader)
print(header)

['헤브론스타국가코드', '현지언어국가명', '영문국가명', '국가언어코드값', '현지언어명', '영문언어명', '현지언어기업명', '통화구분코드', '화폐단위명', '영문기업명', '법인등록번호', '사업자등록번호', '현지언어산업군명', '영문산업군명', '설립일자', '주식시장코드', '현지언어주식시장명', '영문주식시장명', '상장코드', '상장일자', '회계연도', '회계분기', '유동자산금액', '비유동자산금액', '자산총계금액', '유동부채금액', '비유동부채금액', '부채총계금액', '자본총계금액', '부채자본총계금액', '매출액', '매출원가금액', '판매비관리비금액', '이자비용', '기타영업외비용금액', '영업이익손실금액', '이자수익', '기타영업외수익금액', '법인세차감전순이익', '당기순이익/손실금액', '영업활동현금흐름금액', '투자활동현금흐름금액', '재무활동현금흐름금액', '기초현금자산금액', '기말현금자산금액', '부채비율', '영업이익율', '매출액증가율', '영업이익증가율', '당기순이익 증가율', '기업CAGR', '기업 연수', '기업 상태', '기업추가정보값', '작업구분코드', '데이터생성일자', '연계처리상태코드', '연계처리일자']
